In [2]:
import pandas as pd
import numpy as np


In [9]:
test_ad=pd.read_csv("../test/ad.csv")
test_click_log=pd.read_csv("../test/click_log.csv")

In [3]:
train_ad=pd.read_csv("../train_preliminary/ad.csv")
train_click_log=pd.read_csv("../train_preliminary/click_log.csv")
train_user=pd.read_csv("../train_preliminary/user.csv")

In [5]:
train_ad

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213
...,...,...,...,...,...,...
2481130,4445714,3812196,9152,3,56976,248
2481131,4445715,3812197,39063,17,427,\N
2481132,4445716,3812198,39063,17,427,\N
2481133,4445717,3812199,35885,17,427,\N


In [51]:
train_click_log

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1
...,...,...,...,...
30082766,6,30920,228129,1
30082767,57,30920,2361397,1
30082768,26,30920,629802,1
30082769,86,30920,2713031,1


In [52]:
train_user

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1
...,...,...,...
899995,899996,5,1
899996,899997,3,2
899997,899998,4,2
899998,899999,3,1


In [53]:
train_merged=pd.merge(train_ad,train_click_log,on='creative_id')
train_merged=pd.merge(train_merged,train_user,on='user_id')
display(train_merged)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,gender
0,1,1,\N,5,381,78,81,398695,1,4,1
1,68590,63392,\N,18,14681,88,14,398695,1,4,1
2,127337,114621,\N,18,9409,25,3,398695,1,4,1
3,138157,124124,23,2,9065,319,8,398695,1,4,1
4,147066,132027,\N,18,22918,319,1,398695,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
30082766,4263153,3657208,1810,2,61922,319,90,630049,1,3,2
30082767,4291152,3680507,2400,2,57709,246,89,630049,1,3,2
30082768,4374249,3750607,\N,18,10986,6,89,630049,2,3,2
30082769,4376574,3752641,1261,2,10988,6,90,630049,2,3,2


In [56]:
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]
targets=["age","gender"]

In [163]:
from math import log
def cal_idf(df:pd.DataFrame):
    features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]
    num_text=df["user_id"].nunique()
    user_id=df["user_id"].unique()
    for feature in features:
        idf={}
        feature_list=df[feature].unique()
#         cal idf
        for feature_value in feature_list:
            idf[feature_value]=1
            for uid in user_id:
                for each in df[df.user_id==uid][feature]:
                    if each==feature_value:
                        idf[feature_value]+=1
                        break
            idf[feature_value]=log(num_text/idf[feature_value])
#             cal tf and tf_idf
        for uid in user_id:
            tmp_df=df[df.user_id==uid]
            doc_len=tmp_df.size
            tf=tmp_df.groupby(feature)[feature].nunique()
            index=tmp_df.index[0]
            for feature_value in tmp_df[feature]:
                tf_idf=(tf[feature_value]/doc_len)*idf[feature_value]
                if tf_idf==0:
                    print(feature,feature_value)
                df.at[index,feature]=tf_idf
                index+=1
    return df
            

In [222]:
tmpdf=train_merged[train_merged["user_id"]<=10]
tmpdf.info()
display(tmpdf)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 6085617 to 27933244
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   creative_id       256 non-null    int64 
 1   ad_id             256 non-null    int64 
 2   product_id        256 non-null    object
 3   product_category  256 non-null    int64 
 4   advertiser_id     256 non-null    int64 
 5   industry          256 non-null    object
 6   time              256 non-null    int64 
 7   user_id           256 non-null    int64 
 8   click_times       256 non-null    int64 
 9   age               256 non-null    int64 
 10  gender            256 non-null    int64 
dtypes: int64(9), object(2)
memory usage: 24.0+ KB


,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,gender
6085617,5275,6499,\N,18,1263,134,81,8,2,5,1
6085618,14993,16404,\N,18,1903,1,11,8,1,5,1
6085619,15155,16559,1268,2,1295,5,28,8,1,5,1
6085620,63726,59032,1469,2,24721,21,26,8,1,5,1
6085621,69876,64587,26858,3,878,60,43,8,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...
27933240,2729422,2347434,2031,2,20264,238,59,9,1,5,1
27933241,2822758,2426739,2031,2,34428,238,86,9,1,5,1
27933242,2822877,2426842,2031,2,34428,238,85,9,1,5,1
27933243,2936315,2524611,1380,2,26404,238,65,9,2,5,1


In [227]:
def cal_idf_sklearn(df:pd.DataFrame):
    from sklearn.feature_extraction.text import TfidfVectorizer
    features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time"]
    v = TfidfVectorizer(max_df=1.0,min_df=1)
    for feature in features:
        cur=df[feature].astype(str)
#         display(cur)
        x = v.fit_transform(cur)
#         df.drop(feature,axis=1,inplace=True)
        new_df=pd.DataFrame(x.toarray())
        df=pd.concat([df,new_df],axis=1)
    return df
cal_idf_sklearn(tmpdf)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,...,71,72,73,74,75,76,77,78,79,80
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27933240,2729422.0,2347434.0,2031,2.0,20264.0,238,59.0,9.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27933241,2822758.0,2426739.0,2031,2.0,34428.0,238,86.0,9.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27933242,2822877.0,2426842.0,2031,2.0,34428.0,238,85.0,9.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27933243,2936315.0,2524611.0,1380,2.0,26404.0,238,65.0,9.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ValueError: Shape of passed values is (256, 244), indices imply (256, 1)

In [159]:
tmpdf=cal_idf(tmpdf)
display(tmpdf)
tmpdf.describe()

product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
product_id \N
produc

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times,age,gender
6085617,0,0,0,0,0,0.00298597,0,8,0,5,1
6085618,0,0,0,0,0,0.00223372,0,8,0,5,1
6085619,0,0,0.00223372,0,0,0.00223372,0,8,0,5,1
6085620,0,0,0.00298597,0,0,0.00223372,0,8,0,5,1
6085621,0,0,0.00169998,0,0,0.00169998,0,8,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...
27933240,0,0,0.00416496,0,0,0.00162125,0,9,0,5,1
27933241,0,0,0.00416496,0,0,0.00162125,0,9,0,5,1
27933242,0,0,0.00416496,0,0,0.00162125,0,9,0,5,1
27933243,0,0,0.00731563,0,0,0.00162125,0,9,0,5,1


,creative_id,ad_id,product_category,advertiser_id,time,user_id,click_times,age,gender
count,256.0,256.0,256.0,256.0,256.0,256.000000,256.0,256.000000,256.000000
mean,0.0,0.0,0.0,0.0,0.0,5.148438,0.0,6.195312,1.195312
std,0.0,0.0,0.0,0.0,0.0,2.674762,0.0,2.088469,0.397218
min,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,4.000000,1.000000
25%,0.0,0.0,0.0,0.0,0.0,3.000000,0.0,5.000000,1.000000
50%,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,5.000000,1.000000
75%,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,7.000000,1.000000
max,0.0,0.0,0.0,0.0,0.0,10.000000,0.0,10.000000,2.000000
